In [64]:
import Pkg

Pkg.add("HiddenMarkovModels")
Pkg.add("Distributions")
Pkg.add("StableRNGs")
Pkg.add("XLSX")


   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`


In [140]:
using HiddenMarkovModels, Distributions
using LinearAlgebra
using StableRNGs
using XLSX
using DataFrames
using Dates
using Distributions
using Statistics

In [17]:
rng = StableRNG(63);

In [90]:
acwi = DataFrame(XLSX.readtable("../esg/acwi_usd_net_03.04.24.xlsx","acwi_usd_net_full", header=true, infer_eltypes=true))
acwi.Date = Date.(acwi.Date, dateformat"u d, Y")
acwi

Row,Date,ACWI
,Date,Float64
1,1988-01-29,0.0249309
2,1988-02-29,0.0576797
3,1988-03-31,0.0307011
4,1988-04-29,0.0125812
5,1988-05-31,-0.0199201
6,1988-06-30,-0.00119118
7,1988-07-29,0.0183463
8,1988-08-31,-0.0551142
9,1988-09-30,0.0428067


In [91]:
describe(acwi.ACWI)

Summary Stats:
Length:         435
Missing Count:  0
Mean:           0.007221
Std. Deviation: 0.043966
Minimum:        -0.198150
1st Quartile:   -0.017986
Median:         0.012573
3rd Quartile:   0.033632
Maximum:        0.123263
Type:           Float64


In [227]:
#= init_guess = [0.5, 0.5]
trans_guess = [0.6 0.4; 0.3 0.7]
dists_guess = [MvNormal([-0.4, -0.7], I), MvNormal([0.4, 0.7], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess); =#

init_guess = [0.5, 0.5]
trans_guess = [0.9 0.1; 0.9 0.1]
#dists_guess = [MvNormal([-0.1, -0.7], I), MvNormal([0.4, 0.7], I)]
dists_guess = [Normal(-0.07,0.20),Normal(0.14,0.07)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

In [194]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, acwi.ACWI);

In [195]:
last(loglikelihood_evolution)

774.9155810120484

In [104]:
transition_matrix(hmm_est)

2×2 Matrix{Float64}:
 0.953852   0.0461485
 0.0537106  0.946289

In [128]:
obs_distributions(hmm_est)

2-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.0015860352863334213, σ=0.05502964773221741)
 Normal{Float64}(μ=0.013469406460119963, σ=0.025198679554070277)

In [130]:
println("regime 1")
println(mean(obs_distributions(hmm_est)[1])*12)
println(std(obs_distributions(hmm_est)[1])* 12^0.5)
println("regime 2")
println(mean(obs_distributions(hmm_est)[2])*12)
println(std(obs_distributions(hmm_est)[2])* 12^0.5)

regime 1
0.019032423436001054
0.190628291589636
regime 2
0.16163287752143957
0.08729078654259356


In [115]:
acwi_short = acwi[Date(2003,01,01) .<= acwi.Date .<= Date(2024,03,31), :]

Row,Date,ACWI
,Date,Float64
1,2003-01-31,-0.0294357
2,2003-02-28,-0.017938
3,2003-03-31,-0.00431422
4,2003-04-30,0.0886378
5,2003-05-30,0.0574971
6,2003-06-30,0.0187239
7,2003-07-31,0.0218749
8,2003-08-29,0.0233834
9,2003-09-30,0.00607447


In [143]:
println(mean(acwi.ACWI))
println(std(acwi.ACWI))
println(mean(acwi_short.ACWI))
println(std(acwi_short.ACWI))


0.007221256079983577
0.04396581164211372
0.00820031798641316
0.044712591384382244


In [131]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, acwi_short.ACWI);

In [174]:
last(loglikelihood_evolution)

21406.816838901992

In [136]:
typeof(hmm_est)

HMM{Vector{Float64}, Matrix{Float64}, Vector{Normal{Float64}}}

In [132]:
transition_matrix(hmm_est)

2×2 Matrix{Float64}:
 0.932483   0.0675174
 0.0569406  0.943059

In [133]:
obs_distributions(hmm_est)

2-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.001677850881617401, σ=0.05909238647054062)
 Normal{Float64}(μ=0.014143605791660168, σ=0.02345202862629552)

In [135]:
println("regime 1")
println(mean(obs_distributions(hmm_est)[1])*12)
println(std(obs_distributions(hmm_est)[1])* 12^0.5)
println("regime 2")
println(mean(obs_distributions(hmm_est)[2])*12)
println(std(obs_distributions(hmm_est)[2])* 12^0.5)

regime 1
0.020134210579408812
0.20470203141494414
regime 2
0.169723269499922
0.08124021024260716


In [137]:
initialization(hmm_est)

2-element Vector{Float64}:
 1.0
 5.644524055265908e-50

In [216]:
simulated_ACWI = Matrix(CSV.read("../sb_gbi/equity_scenarios.csv",DataFrame))[1:1000,:]

1000×50 Matrix{Float64}:
  0.183608   0.040727   0.130638  …   0.310636   0.259417   0.106576
  0.057796  -0.015669   0.01473       0.181475   0.038817   0.09112
 -0.042034   0.071062   0.211914      0.271774   0.03051    0.014144
  0.048233   0.002585   0.069304      0.161997   0.150916  -0.458586
 -0.097277   0.278737   0.249458      0.054621   0.20249    0.148626
  0.168582   0.058298   0.174671  …  -0.057467  -0.037438   0.030723
  0.096997   0.214256  -0.246285      0.148997   0.192833   0.181174
  0.254872   0.054849   0.014113      0.13077    0.066608  -0.076274
  0.263792   0.213822   0.260164      0.309639   0.022386   0.268776
  0.065249  -0.206639   0.347267      0.165357   0.180862   0.298468
  ⋮                               ⋱                        
  0.313924   0.116269   0.06669       0.226258   0.172806   0.186851
  0.296246  -0.045214   0.231189      0.156628   0.218297  -0.019684
 -0.071925  -0.124002  -0.197512      0.287149  -0.01889    0.176335
  0.184442   0.2505

In [217]:
size(simulated_ACWI)

(1000, 50)

In [218]:
simulated_ACWI = [row for row in eachrow(simulated_ACWI)]

1000-element Vector{SubArray{Float64, 1, Matrix{Float64}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}}:
 [0.183608, 0.040727, 0.130638, 0.167958, -0.178384, 0.032183, 0.122386, -0.049013, -0.230813, -0.230181  …  0.005541, 0.204203, 0.084194, -0.195554, 0.164883, 0.105274, 0.357969, 0.310636, 0.259417, 0.106576]
 [0.057796, -0.015669, 0.01473, 0.15795, -0.133039, -0.024649, -0.002556, 0.101583, 0.042812, 0.170594  …  0.049197, 0.108522, 0.101433, 0.089037, -0.170575, 0.116987, 0.206452, 0.181475, 0.038817, 0.09112]
 [-0.042034, 0.071062, 0.211914, 0.19843, 0.246116, 0.020742, 0.222028, 0.107744, 0.092587, -0.058472  …  0.102718, -0.002485, 0.148345, 0.101275, 0.009295, -0.09912, 0.207978, 0.271774, 0.03051, 0.014144]
 [0.048233, 0.002585, 0.069304, -0.053078, -0.156461, 0.279966, 0.010687, -0.049629, -0.09302, 0.076925  …  0.148934, 0.199687, 0.071429, 0.031693, -0.138951, -0.069515, 0.008906, 0.161997, 0.150916, -0.458586]
 [-0.097277, 0.278737, 0.249458, 0.093802, 0.071716, 0.

In [234]:
seq_ends = cumsum(length.(simulated_ACWI))

1000-element Vector{Int64}:
    50
   100
   150
   200
   250
   300
   350
   400
   450
   500
     ⋮
 49600
 49650
 49700
 49750
 49800
 49850
 49900
 49950
 50000

In [233]:
seq_end = cumsum(size(simulated_ACWI,2) for s in 1:size(simulated_ACWI,1))
seq_end

1000-element Vector{Int64}:
    1
    2
    3
    4
    5
    6
    7
    8
    9
   10
    ⋮
  992
  993
  994
  995
  996
  997
  998
  999
 1000

In [212]:
length(simulated_ACWI)

50000

In [222]:
seq_concat = reduce(vcat, simulated_ACWI)

50000-element Vector{Float64}:
  0.183608
  0.040727
  0.130638
  0.167958
 -0.178384
  0.032183
  0.122386
 -0.049013
 -0.230813
 -0.230181
  ⋮
 -0.167158
 -0.16384
  0.079
  0.037215
 -0.039934
 -0.097223
  0.129483
  0.165093
  0.100469

In [223]:
typeof(seq_end)

Vector{Int64} (alias for Array{Int64, 1})

In [235]:
hmm_est, _ = baum_welch(hmm_guess, seq_concat;seq_end);

MethodError: MethodError: no method matching baum_welch(::HMM{Vector{Float64}, Matrix{Float64}, Vector{Normal{Float64}}}, ::Vector{Float64}, ::FillArrays.Fill{Nothing, 1, Tuple{Base.OneTo{Int64}}}; seq_end::Vector{Int64})

Closest candidates are:
  baum_welch(::AbstractHMM, ::AbstractVector, ::AbstractVector; seq_ends, atol, max_iterations, loglikelihood_increasing) got unsupported keyword argument "seq_end"
   @ HiddenMarkovModels C:\Users\matsz\.julia\packages\HiddenMarkovModels\y95cy\src\inference\baum_welch.jl:54
  baum_welch(::AbstractHMM, ::AbstractVector; ...)
   @ HiddenMarkovModels C:\Users\matsz\.julia\packages\HiddenMarkovModels\y95cy\src\inference\baum_welch.jl:54


In [231]:
obs_distributions(hmm_est)

2-element Vector{Normal{Float64}}:
 Normal{Float64}(μ=0.03021809892555822, σ=0.1671953920109272)
 Normal{Float64}(μ=0.1479416812996293, σ=0.11580532147295382)

In [232]:
transition_matrix(hmm_est)

2×2 Matrix{Float64}:
 0.791468  0.208532
 0.288195  0.711805

In [214]:
nb_seqs = 100
long_obs_seqs = [last(rand(rng, hmm, rand(rng, 100:200))) for k in 1:nb_seqs];
typeof(long_obs_seqs)

Vector{Vector{Vector{Float64}}} (alias for Array{Array{Array{Float64, 1}, 1}, 1})

In [213]:
long_obs_seq_concat = reduce(vcat, long_obs_seqs)
typeof(long_obs_seq_concat)

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

In [207]:
seq_ends = cumsum(length.(long_obs_seqs))
seq_ends

100-element Vector{Int64}:
   147
   312
   512
   624
   780
   902
  1050
  1208
  1371
  1569
     ⋮
 14155
 14266
 14385
 14521
 14660
 14804
 14987
 15093
 15261

In [225]:
init_guess = [0.5, 0.5]
trans_guess = [0.6 0.4; 0.3 0.7]
dists_guess = [MvNormal([-0.4, -0.7], I), MvNormal([0.4, 0.7], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

In [204]:
hmm_est_concat, _ = baum_welch(hmm_guess, long_obs_seq_concat; seq_ends);